In [58]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./Data/VideoStarted.csv')

In [3]:
df.head()

,UserId,Date,Minute_Of_Day,Second,State,Genre,Category,ProgramType,Country,Device,OS,VideoId
0,0280dfdd112732a3ac12b12dc770b7af,20170106,0,8,35,Romance,vod,Movies,1,1,2,36a27b379622f342ec87f9aafadb8f94
1,435d41ae019cb8db785483793859c9a8,20170106,0,28,35,Anime,vod,TV Shows,1,1,1,a4d9b88c7ed63d723c70b358a857719c
2,1faf0ce0b98e02e1568702f516f01a78,20170106,0,36,35,Drama,vod,TV Shows,1,1,1,806660cb47633263a24bbc53238a9a53
3,3ec691b9d2b5d53ef965fe59b1900b30,20170106,0,48,35,Drama,vod,TV Shows,1,2,2,385114825a85d6878e7a0978f9ba5546
4,ff230d487a7139b65f33b54a4cbd2d9e,20170106,0,56,35,Comedy,vod,Movies,1,1,1,368a480ec0ae105aee8320dd93483e39


In [64]:
df.shape

(1796525, 12)

In [67]:
len(df['UserId'].unique())

165093

In [68]:
len(df['VideoId'].unique())

3550

In [70]:
x =  df[:100].groupby(['UserId', 'VideoId'])
x = x.first()
x

Date  \
UserId                           VideoId                                      
0280dfdd112732a3ac12b12dc770b7af 36a27b379622f342ec87f9aafadb8f94  20170106   
04c1e14fd54a06c0d4fe98cf0cb8b04c 806660cb47633263a24bbc53238a9a53  20170106   
057212548aff678c01fe2a55224cc2cc f3ecc88c6e53a990a75bd7e9b7ddb06e  20170106   
079ec44549c738db2d55ff44b5b139f3 66ea27601d482d1857f6977450b2dc1c  20170106   
087c741c167a352d07822fcab58046e5 2e47c28c5a2f4ac65315d0229447fdf8  20170106   
0d164b3342b7c442b8a5cfc8a608bee1 afb85ac1bc0bb04bd7227ff5bf925bad  20170106   
                                 d0c64e942ff427750850e49894ec48fe  20170106   
0d2717ba754737c78430788ed77a1db7 675548e120ccb01387cc3ca9fed870a9  20170106   
13d9f55e297d7d9c219fbe9085e1006c 8b6076b28243980a22f496bb127f3aec  20170106   
16455c25d7a8af4084973ec08d320cd5 99e672aa585e8af17d01da8b839b6d18  20170106   
19859f9b1e56c687b010391bc6670a76 57622239c0d0c605eb50e6923ebabb8a  20170106   
1f9be9226d4393fff4b8d6b94e54c4dd 2dd067e7a439ac5a433bb7aa7eec6b7b  20170106   
                                 a4a773531e35757badfe759f6d69e7a7  20170106   
                                 aa6dab080812d9e8b8e6b8f07c7a399d  20170106   
1faf0ce0b98e02e1568702f516f01a78 806660cb47633263a24bbc53238a9a53  20170106   
25e756b60e551802a441a50e794eff87 21f099b73c49eb58c49371abe5cfe63e  20170106   
2956ff6e5c89beb4bd9e9590ba5596b4 67197f3b5830a66b117539268da239eb  20170106   
32426aebb390a2561d793461a58b0ee8 1e16901cf325d640453b0d9279098803  20170106   
                                 475774a85b02f70fbd08face24d6cec8  20170106   
                                 e6ea7e407888ec088e91bafe475c91d0  20170106   
32d8926c81149eb84ca5af5031a668a0 806660cb47633263a24bbc53238a9a53  20170106   
335e967c2d1836de8e059d929c1a1eb3 58496b4becdf567f1b9f02943a6489a2  20170106   
                                 993c5767292cd8922c422b348d582a5c  20170106   
34a84047a6de70021be428b4ee3152bc e6ea7e407888ec088e91bafe475c91d0  20170106   
3ec691b9d2b5d53ef965fe59b1900b30 385114825a85d6878e7a0978f9ba5546  20170106   
42679123b0cc545eea930f1406704810 ef2f8cc077424405c8d09031ad76ef92  20170106   
435d41ae019cb8db785483793859c9a8 a4d9b88c7ed63d723c70b358a857719c  20170106   
4a48b6f152717235733e4bd48e5a3449 dad5c43748b229667804b2bcd8a40bf5  20170106   
                                 e6ea7e407888ec088e91bafe475c91d0  20170106   
4a9937fc586eddbdee50af81dec74364 66674d25b31f8b9e39389c3dcc94c9a0  20170106   
...                                                                     ...   
b0f1608f6f08dc67df769bd755fb1814 c0883669ac53bc724949ee0cf54457a9  20170106   
                                 dfaf22b3245593f379be6579faa7535f  20170106   
b2537964c1c6812227164f2d21e4f056 2dd067e7a439ac5a433bb7aa7eec6b7b  20170106   
                                 f88d90265ada11b808fc25417e78e3b5  20170106   
b886b6f41c5e9667793644db95a6fe6a e6ea7e407888ec088e91bafe475c91d0  20170106   
b8fd5e70eb720b258b6ad484d382d055 e6ea7e407888ec088e91bafe475c91d0  20170106   
                                 ef2f8cc077424405c8d09031ad76ef92  20170106   
ba53135c9063649e4a25138e81606198 82a7154f05d54bd1a29ec73b76289870  20170106   
c053cb4e67c4dd27bc82ec66a8f4118a 1fe4df4a97ac85e903edf8009b10879a  20170106   
c91bd3afdb4485660be836b73e534873 21f099b73c49eb58c49371abe5cfe63e  20170106   
ca601ff304bf0c0836f8a59f7751b8de 21526ec273ad7b6d230822f32d685fd1  20170106   
cc298af21bdb025746e0ee98148091ba ef2f8cc077424405c8d09031ad76ef92  20170106   
d00fdd18d39a97eb4e773ffebda87912 be2885a06706b121ae3b731d416c0160  20170106   
d474ce5957c78efaa070f8266791060c e6ea7e407888ec088e91bafe475c91d0  20170106   
d4984df3c20408ce64fdfed2ebd62fd2 a4a773531e35757badfe759f6d69e7a7  20170106   
                                 e6ea7e407888ec088e91bafe475c91d0  20170106   
                                 ef2f8cc077424405c8d09031ad76ef92  20170106   
dc93d200565e5bf6e6fc55adb1c5ba06 e231185629db5a1bc134cd088ff190e5  20170106   
                                 febe578c5

In [86]:
user_video_mapper = dict()

def some_function(row):
    user_id = 'user_' + row['UserId']
    video_id = 'video_' + row['VideoId']
    
    if user_id not in user_video_mapper:
        user_video_mapper[user_id] = dict()
    
    video_info_of_user = user_video_mapper[user_id]
    
    if video_id not in video_info_of_user:
        video_info_of_user[video_id] = 1
    
    user_video_mapper[user_id] = video_info_of_user
    
df.apply(some_function, axis=1)
user_video_mapper

{'user_0280dfdd112732a3ac12b12dc770b7af': {'video_36a27b379622f342ec87f9aafadb8f94': 1},
 'user_435d41ae019cb8db785483793859c9a8': {'video_a4d9b88c7ed63d723c70b358a857719c': 1},
 'user_1faf0ce0b98e02e1568702f516f01a78': {'video_806660cb47633263a24bbc53238a9a53': 1,
  'video_e6ea7e407888ec088e91bafe475c91d0': 1,
  'video_29145661d92ca9ba126436374184d043': 1,
  'video_832a5de26fdcd68abd952a1d26af5b6a': 1,
  'video_9e7eb053c618e8c13289c33dccf21a28': 1,
  'video_8e52d2ec8e77c7df3b0aba1de3beedd9': 1,
  'video_4b894a4f2ed090ead1ad053905ac5701': 1,
  'video_74843ecae1130385a03df9bfa138af0f': 1,
  'video_77ddf91bdbfa67161f98ad83d0161e69': 1},
 'user_3ec691b9d2b5d53ef965fe59b1900b30': {'video_385114825a85d6878e7a0978f9ba5546': 1},
 'user_ff230d487a7139b65f33b54a4cbd2d9e': {'video_368a480ec0ae105aee8320dd93483e39': 1,
  'video_99ca08ae5c8f6d778de6e1d7c1addc17': 1,
  'video_6b675d876182201fa22887e568c956b7': 1,
  'video_be868cd2719f11a6d7938adc6ed6ab4a': 1,
  'video_5c494194707c59c1742bebd9aad1b8

In [87]:
len(user_video_mapper.keys())

165093

In [88]:
user_video_df = pd.DataFrame(user_video_mapper).T

In [89]:
user_video_df.head()

,video_001f728d69e49aad15ecaecddb429906,video_001f7cd950bf1bf42634df4dc5692e5d,video_002510424dad09c3575f676dba07c52a,video_0039f67ba5c9538b04f96e8c0cfd5119,video_004cf12035df1e5018577003cf61bfec,video_0054ab07962c6e0ae6d9ed17fd05444e,video_005f0e9f7b0f110634f3b25711d8e4c2,video_0069536be5ac83d7ab75e355c1509f9d,video_00b009802fda66ab98ac10a5935200aa,video_00b4524f6d283a38a2bad3e302474597,...,video_ff89b2ddd4ffc2392ead75ded4f45192,video_ff9b70b0e80552004d42ac19419ab1de,video_ff9be26eb4f704ace04f9af9fd16f19e,video_ffa23e230497f2265c1780bf295c3afe,video_ffd69222950dd0d36a9c3bf75a442030,video_ffe15f453c0bd4ac1a450687f118c78a,video_ffe256d11a3fdecd58772e7dae50cc60,video_ffe7044026752991081c5664e12fe977,video_ffefd73588aa97b71067b33847ab1ef8,video_fffc6e567925a523e2943a711881e5ce
user_0280dfdd112732a3ac12b12dc770b7af,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_435d41ae019cb8db785483793859c9a8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_1faf0ce0b98e02e1568702f516f01a78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_3ec691b9d2b5d53ef965fe59b1900b30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_ff230d487a7139b65f33b54a4cbd2d9e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
user_video_df = user_video_df.replace(np.nan, 0)

In [91]:
user_video_df.shape

(165093, 3550)

In [92]:
user_video_df.describe()

,video_001f728d69e49aad15ecaecddb429906,video_001f7cd950bf1bf42634df4dc5692e5d,video_002510424dad09c3575f676dba07c52a,video_0039f67ba5c9538b04f96e8c0cfd5119,video_004cf12035df1e5018577003cf61bfec,video_0054ab07962c6e0ae6d9ed17fd05444e,video_005f0e9f7b0f110634f3b25711d8e4c2,video_0069536be5ac83d7ab75e355c1509f9d,video_00b009802fda66ab98ac10a5935200aa,video_00b4524f6d283a38a2bad3e302474597,...,video_ff89b2ddd4ffc2392ead75ded4f45192,video_ff9b70b0e80552004d42ac19419ab1de,video_ff9be26eb4f704ace04f9af9fd16f19e,video_ffa23e230497f2265c1780bf295c3afe,video_ffd69222950dd0d36a9c3bf75a442030,video_ffe15f453c0bd4ac1a450687f118c78a,video_ffe256d11a3fdecd58772e7dae50cc60,video_ffe7044026752991081c5664e12fe977,video_ffefd73588aa97b71067b33847ab1ef8,video_fffc6e567925a523e2943a711881e5ce
count,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000,...,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000,165093.000000
mean,0.000157,0.000775,0.000479,0.000321,0.000218,0.000030,0.004361,0.003471,0.000448,0.000654,...,0.000164,0.000818,0.000333,0.000073,0.000121,0.000254,0.011727,0.001133,0.000006,0.000315
std,0.012548,0.027834,0.021870,0.017915,0.014765,0.005503,0.065895,0.058811,0.021167,0.025569,...,0.012787,0.028584,0.018249,0.008525,0.011006,0.015948,0.107654,0.033637,0.002461,0.017745
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
